In [1]:
from pyspark import SQLContext
sqlCtx=SQLContext(sc)
coursesdata = sqlContext.read.csv('/FileStore/tables/iyszh6vb1493262685424/courses.csv', header=True)
studentAssessment= sqlContext.read.csv('/FileStore/tables/iyszh6vb1493262685424/studentAssessment.csv', header=True)
StudentInfodata = sqlContext.read.csv('/FileStore/tables/iyszh6vb1493262685424/studentInfo.csv', header=True).fillna('na')
studentRegistration = sqlContext.read.csv('/FileStore/tables/iyszh6vb1493262685424/studentRegistration.csv',header=True)
studentVle = sqlContext.read.csv('/FileStore/tables/ledsvqke1493512048163/studentVle.csv',header=True)
#StudentInfodata.show()

StudentInfodata.registerTempTable("Students")
sqlCtx.sql("SELECT COUNT(distinct id_student) FROM Students ").show()

In [2]:
import numpy as np
import pandas as pd

#studentcourseassesment 
StudentInfodata.registerTempTable("Students")
coursesdata.registerTempTable("Courses")
studentAssessment.registerTempTable("StudentAsses")
                              
sqlCtx.sql(""" select s.* from Students AS s JOIN Courses AS c ON c.code_module == s.code_module AND c.code_presentation == s.code_presentation """).registerTempTable("studentcourse") #207319 


sqlCtx.sql(""" SELECT sa.id_assessment,sa.date_submitted,sa.is_banked,CASE WHEN sa.score IS NULL THEN 0 ELSE sa.score END AS score, sc.code_module ,sc.code_presentation,sc.id_student,sc.gender,sc.region,sc.highest_education,sc.imd_band,sc.age_band,sc.num_of_prev_attempts,sc.studied_credits,sc.disability,
 CASE WHEN sc.final_result =='Fail' THEN 0 WHEN sc.final_result == 'Pass' THEN 1 ELSE 2 END AS final_result FROM studentcourse AS sc JOIN StudentAsses AS sa ON sc.id_student = sa.id_student WHERE sc.final_result !='Withdrawn' """).registerTempTable("studentcourseassesment") #207319


studentRegistration.registerTempTable("studentReg")

sqlCtx.sql(""" SELECT CASE WHEN sr.date_registration IS NULL THEN 0 ELSE sr.date_registration END AS date_registration ,CASE WHEN sr.date_unregistration IS NULL THEN 0 ELSE sr.date_unregistration END AS date_unregistration,sa.*  FROM studentReg as sr JOIN studentcourseassesment AS sa ON sr.id_student = sa.id_student AND sr.code_presentation = sa.code_presentation   """).registerTempTable("studentcourassesreg") #count =207319

import pandas as pd
studentVle.registerTempTable("studentvle")
sqlCtx.sql("SELECT code_module,code_presentation,id_student,SUM(sum_click) AS sum_click FROM studentvle GROUP BY code_module,code_presentation,id_student").registerTempTable("studentvleupdated") 

sqlCtx.sql(""" SELECT CASE WHEN sr.date_registration IS NULL THEN 0 ELSE sr.date_registration END AS date_registration ,CASE WHEN sr.date_unregistration IS NULL THEN 0 ELSE sr.date_unregistration END AS date_unregistration,sa.*  FROM studentReg as sr JOIN studentcourseassesment AS sa ON sr.id_student = sa.id_student  AND sr.code_presentation = sa.code_presentation   """).registerTempTable("studentcourassesreg") #count

outputcheck=sqlCtx.sql("SELECT svle.sum_click,sr.* FROM studentvleupdated AS svle JOIN studentcourassesreg as sr ON svle.id_student = sr.id_student AND svle.code_presentation = sr.code_presentation  ")

display(outputcheck)
